In [10]:
# 신경망 모델 구성하기

In [8]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 학습을 위한 장치 얻기

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## 클래스 정의하기
- 신경망 모델을 `nn.Module`의 하위클래스로 정의
- `__init__`에서 신경망 계층들을 초기화
- `nn.Module`을 상속받은 모든 클래스는 `forward` 메소드에서, 입력 데이터에 대한 연산을 구현 

In [21]:
class NueralNetwork(nn.Module):
    def __init__(self):
        super(NueralNetwork, self).__init__()
        self.flatten = nn.Flatten() # 이미지를 1차원으로 펼쳐줌
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


- 모델 구조 출력

In [22]:
model = NueralNetwork().to(device)
print(model)

NueralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [23]:
X = torch.rand(3, 32, 32, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8, 8, 5])


## 기본적인 합성곱 신경망 구성 방법
 - `torch.nn.conv2d` 참고 링크 => [홈페이지](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

In [32]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # 첫번째 2D 합성곱 계층
      # 1개의 입력 채널(이미지)을 받아들이고, 사각 커널 사이즈가 3인 32개의 합성곱 특징들을 출력합니다.
      # (in_channels, out_channels, kernel_size, stride, padding='valid', 'same' or int)
      self.conv1 = nn.Conv2d(1, 32, 3, 1, 'same')
      # 두번째 2D 합성곱 계층
      # 32개의 입력 게층을 받아들이고, 사각 커널 사이즈가 3인 64개의 합성곱 특징을 출력합니다.
      self.conv2 = nn.Conv2d(32, 64, 3, 1)

      # 인접한 픽셀들은 입력 확률에 따라 모두 0 값을 가지거나 혹은 모두 유효한 값이 되도록 만듭니다.
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)

      # 첫번째 fully connected layer
      self.fc1 = nn.Linear(9216, 128)
      # 10개의 라벨을 출력하는 두번째 fully connected layer
      self.fc2 = nn.Linear(128, 10)

my_nn = Net()
print(my_nn)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [25]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(9216, 128)
      self.fc2 = nn.Linear(128, 10)

    # x는 데이터를 나타냅니다.
    def forward(self, x):
      # 데이터가 conv1을 지나갑니다.
      x = self.conv1(x)
      # x를 ReLU 활성함수(rectified-linear activation function)에 대입합니다.
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      # x에 대해서 max pooling을 실행합니다.
      x = F.max_pool2d(x, 2)
      # 데이터가 dropout1을 지나갑니다.
      x = self.dropout1(x)
      # start_dim=1으로 x를 압축합니다.
      x = torch.flatten(x, 1)
      # 데이터가 fc1을 지나갑니다.
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # x에 softmax를 적용합니다.
      output = F.log_softmax(x, dim=1)
      return output

In [27]:
import torch.nn.functional as F

# 임의의 28x28 이미지로 맞춰줍니다.
random_data = torch.rand((1, 1, 28, 28))

my_nn = Net()
result = my_nn(random_data)
print (result)

tensor([[-2.4453, -2.3968, -2.3191, -2.3904, -2.1992, -2.3121, -2.1617, -2.3920,
         -2.2905, -2.1664]], grad_fn=<LogSoftmaxBackward>)
/usr/local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
